# Use scikit-learn to recognize hand-written digits with `ibm-watsonx-ai`

This notebook contains steps and code to demonstrate how to persist and deploy a locally trained scikit-learn model in the watsonx.ai with [ibm-watsonx-ai](https://pypi.python.org/pypi/ibm-watsonx-ai) library available in PyPI repository. This notebook introduces commands for getting a model and training data, persisting the model, deploying it, scoring it, updating it, and redeploying it.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goals

The learning goals of this notebook are:

-  Train an scikit-learn model
-  Persist the trained model in the watsonx.ai
-  Deploy the model for online scoring using the client library
-  Score sample records using the client library


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Explore data and create a scikit-learn model](#train)
3. [Persist the externally created scikit model](#upload)
4. [Deploy and score](#deploy)
5. [Clean up](#cleanup)
6. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, contact your Cloud Pak for Data administrator and ask for your account credentials.


### Install dependencies
**Note:** `ibm-watsonx-ai` documentation can be found <a href="https://ibm.github.io/watsonx-ai-python-sdk/index.html" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
%pip install wget | tail -n 1
%pip install "scikit-learn==1.6.1" | tail -n 1
%pip install -U ibm-watsonx-ai | tail -n 1

#### Define credentials

Authenticate the watsonx.ai Runtime service on IBM Cloud Pak for Data. You need to provide the **admin's** `username` and the platform `url`.

In [ ]:
username = "PASTE YOUR USERNAME HERE"
url = "PASTE THE PLATFORM URL HERE"

Use the **admin's** `api_key` to authenticate watsonx.ai Runtime services:

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    username=username,
    api_key=getpass.getpass("Enter your watsonx.ai API key and hit enter: "),
    url=url,
    instance_id="openshift",
    version="5.2",
)

Alternatively you can use the **admin's** `password`:

In [3]:
import getpass
from ibm_watsonx_ai import Credentials

if "credentials" not in locals() or not credentials.api_key:
    credentials = Credentials(
        username=username,
        password=getpass.getpass("Enter your watsonx.ai password and hit enter: "),
        url=url,
        instance_id="openshift",
        version="5.2",
    )

#### Create `APIClient` instance

In [4]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials)

### Working with spaces

First of all, you need to create a space that will be used for your work. If you do not have a space, you can use `{PLATFORM_URL}/ml-runtime/spaces?context=icp4data` to create one.

- Click New Deployment Space
- Create an empty space
- Go to space `Settings` tab
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [ ]:
space_id = "PASTE YOUR SPACE ID HERE"

You can use the `list()` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set **space** which you will be using.

In [6]:
client.set.default_space(space_id)

'SUCCESS'

<a id="train"></a>
## 2. Explore data and create an scikit-learn model
In this section, you will prepare and train a handwritten digits model using the scikit-learn library.

### 2.1 Explore data

As the first step, you will load the data from scikit-learn sample datasets and perform basic exploration.

In [7]:
from sklearn import datasets

digits = datasets.load_digits()

Loaded dataset consists of 8x8 pixels images of hand-written digits.

Let's display first digit data and label using **data** and **target**.

In [8]:
print(digits.data[0].reshape((8, 8)))

[[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]]


In [9]:
digits.target[0]

np.int64(0)

In the next step, you will count data examples.

In [10]:
samples_count = len(digits.images)
print(f"Number of samples: {samples_count}")

Number of samples: 1797


### 2.2. Create a scikit-learn model

**Prepare data**

In this step, you'll split your data into three datasets:
- train
- test
- score

In [11]:
train_data = digits.data[: int(0.7 * samples_count)]
train_labels = digits.target[: int(0.7 * samples_count)]

test_data = digits.data[int(0.7 * samples_count) : int(0.9 * samples_count)]
test_labels = digits.target[int(0.7 * samples_count) : int(0.9 * samples_count)]

score_data = digits.data[int(0.9 * samples_count) :]

print(f"Number of training records: {len(train_data)}")
print(f"Number of testing records : {len(test_data)}")
print(f"Number of scoring records : {len(score_data)}")

Number of training records: 1257
Number of testing records : 360
Number of scoring records : 180


**Create pipeline**

Standardize features by removing the mean and scaling to unit variance.

In [12]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

Next, define estimators you want to use for classification. Support Vector Machines (SVM) with the radial basis function as kernel is used in the following example.

In [13]:
from sklearn import svm

clf = svm.SVC(kernel="rbf")

Let's build the pipeline now. This pipeline consists of a transformer and an estimator.

In [14]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([("scaler", scaler), ("svc", clf)])

**Train model**

Now, you can train your SVM model by using the previously defined **pipeline** and **train data**.

In [15]:
model = pipeline.fit(train_data, train_labels)

**Evaluate model**

You can check your **model quality** now. To evaluate the model, use **test data**.

In [16]:
from sklearn import metrics

predicted = model.predict(test_data)

print(
    "Evaluation report:",
    metrics.classification_report(test_labels, predicted),
    sep="\n\n",
)

Evaluation report:

              precision    recall  f1-score   support

           0       1.00      0.97      0.99        37
           1       0.97      0.97      0.97        34
           2       1.00      0.97      0.99        36
           3       1.00      0.94      0.97        35
           4       0.78      0.97      0.87        37
           5       0.97      0.97      0.97        38
           6       0.97      0.86      0.91        36
           7       0.92      0.97      0.94        35
           8       0.91      0.89      0.90        35
           9       0.97      0.92      0.94        37

    accuracy                           0.94       360
   macro avg       0.95      0.94      0.95       360
weighted avg       0.95      0.94      0.95       360



You can tune your model now to achieve better accuracy. For simplicity, tuning section is omitted.

<a id="upload"></a>
## 3. Persist the locally created scikit-learn model

In this section, you will learn how to store your model in the Watson Machine Learning repository by using the IBM watsonx.ai SDK.

### 3.1: Publish model

#### Publish the model in the Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [17]:
sofware_spec_id = client.software_specifications.get_id_by_name("runtime-25.1-py3.12")

In [18]:
metadata = {
    client.repository.ModelMetaNames.NAME: "Scikit model",
    client.repository.ModelMetaNames.TYPE: "scikit-learn_1.6",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sofware_spec_id,
}

published_model = client.repository.store_model(
    model=model,
    meta_props=metadata,
    training_data=train_data,
    training_target=train_labels,
)

### 3.2: Get model details

In [19]:
import json

published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "name": "Scikit model",
    "space_id": "8a13841b-df99-4b4d-bf2a-161ad2e33980",
    "resource_key": "fe4d8ae4-eb9a-40a5-88f2-244ceea804f7",
    "id": "4979c0ed-b412-4f73-a293-f0b952e2c988",
    "created_at": "2025-05-15T07:38:26Z",
    "rov": {
      "member_roles": {
        "1000331001": {
          "user_iam_id": "1000331001",
          "roles": [
            "OWNER"
          ]
        }
      }
    },
    "owner": "1000331001"
  },
  "entity": {
    "software_spec": {
      "id": "f47ae1c3-198e-5718-b59d-2ea471561e9e"
    },
    "type": "scikit-learn_1.6",
    "training_data_references": [
      {
        "id": "1",
        "type": "fs",
        "connection": null,
        "location": {},
        "schema": {
          "id": "1",
          "type": "ndarray",
          "fields": [
            {
              "name": "f0",
              "type": "float",
              "nullable": false
            },
            {
              "name": "f1",
              "type":

### 3.3 Get all models

In [20]:
models_details = client.repository.list_models()

<a id="deploy"></a>
## 4. Deploy and score

In this section, you will learn how to create online scoring and to score a new data record by using the IBM watsonx.ai SDK.

### 4.1: Create a model deployment

#### Create an online deployment for the published model

In [21]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '4979c0ed-b412-4f73-a293-f0b952e2c988' started

######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='924ce586-edfc-4e3f-b07b-587935859f5e'
-----------------------------------------------------------------------------------------------




**Note**: Here we use the deployment url saved in the published_model object. In the next section, we show how to retrieve the deployment url from the Watson Machine Learning instance.

In [22]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [23]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

You can also list existing deployments.

In [24]:
client.deployments.list()

,ID,NAME,STATE,CREATED,ARTIFACT_TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,924ce586-edfc-4e3f-b07b-587935859f5e,Deployment of scikit model,ready,2025-05-15T07:38:43.081Z,model,supported,


### 4.2: Get deployment details

In [25]:
client.deployments.get_details(deployment_id)

### 4.3: Score

You can use the following method to perform a test scoring request against the deployed model.

**Action**: Prepare scoring payload with records to score.

In [26]:
score_0 = list(score_data[0])
score_1 = list(score_data[1])

In [27]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use ``client.deployments.score()`` method to run scoring.

In [28]:
predictions = client.deployments.score(deployment_id, scoring_payload)

In [29]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "fields": [
        "prediction"
      ],
      "values": [
        [
          5
        ],
        [
          4
        ]
      ]
    }
  ]
}


<a id="cleanup"></a>
## 5. Cleanup 

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

follow the steps listed in this sample [notebook](https://github.com/IBM/watsonx-ai-samples/blob/master/cpd5.2/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps

You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. 
 
Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer

Copyright © 2020-2025 IBM. This notebook and its source code are released under the terms of the MIT License.